# how to build hakken 


> i will explain how i build the hakken: coding agent. we will cover some fundamental on how you can build agent and then will build some of the features of hakken not all tho but some.

## outline 

- we wil start from simple llm tool call 
- then we will build react agent 
- then we will add some features


> # part-1 : llm tool call 

In [4]:
## imports 
import json, requests
from openai import OpenAI
from tavily import TavilyClient # for web search 
import os 
from dotenv import load_dotenv
load_dotenv()

True

- you can get api for llm (free): [link](https://openrouter.ai/z-ai/glm-4.5-air:free)
- and for our web search tool (free): [link](https://www.tavily.com/)


> ### model : llm 

In [43]:
## lets define our llm 

#get the keys from env 
openrouter_key = os.getenv("OPENROUTER_API_KEY")

openai_client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=openrouter_key,
)



> ### tool description 

In [ ]:
tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
from typing import Literal

def internet_search(
    query: str,
    max_results: int = 5,
    topic: Literal["general", "news", "finance"] = "general",
    include_raw_content: bool = False,
):
    search_docs = tavily_client.search(
        query,
        max_results=max_results,
        include_raw_content=include_raw_content,
        topic=topic,
    )
    return search_docs

tools = [
    {
        "type": "function",
        "function": {
            "name": "internet_search",
            "description": "Search the internet for current information, news, and research. Can search across general topics, news, or finance-specific content. Useful for finding up-to-date information, research developments, tech news, and current events.",
            "parameters": {
                "type": "object",
                "properties": {
                    "search_terms": {
            "type": "array",
            "items": {"type": "string"},
            "description": "List of search terms to find books"
          }
                    
                },
                "required": ["query"]
            }
        }
    }
]



In [44]:
task = "can you search new ai agent trends in 2025 ?"

messages = [
  {
    "role": "system",
    "content": "You are a helpful assistant."
  },
  {
    "role": "user",
    "content": task, ## our query 
  }
]

> ###  lets run without tool call; just simple llm call 

In [45]:
model_name = os.getenv("OPENAI_MODEL")

request_1 = {
    "model":model_name,
    "tools": tools,
    "messages": messages
}
response_1 = openai_client.chat.completions.create(**request_1).choices[0].message
print(response_1)


ChatCompletionMessage(content="\nI'll search for the latest AI agent trends and developments expected in 2025.\n", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_sv7GX3ElTwOlBOl_vgFeZQ', function=Function(arguments='{"query": "new AI agent trends 2025", "topic": "general"}', name='internet_search'), type='function', index=0)], reasoning='\nThe user is asking me to search for new AI agent trends in 2025. This is a request for current information about AI agent developments and trends for the upcoming year. I should use the internet_search function to find relevant information about this topic.\n\nI\'ll use:\n- query: "new AI agent trends 2025" \n- topic: "general" (since this is about technology trends rather than specifically news or finance)\n- max_results: I\'ll use the default of 5 to get a good overview\n- include_raw_content: I\'ll keep this as false to get cleaner results\n\nThe query sh

> ### okay its working 

In [47]:
messages.append(response_1)
for tool_call in response_1.tool_calls:
    args = json.loads(tool_call.function.arguments)
    tool_result = internet_search(**args)
    messages.append({
        "role": "tool",
        "tool_call_id": tool_call.id,
        "name": tool_call.function.name,
        "content": json.dumps(tool_result)
    })

In [49]:
request_2 = {
        "model": model_name,
        "messages": messages,
        "tools": tools
    }
response_2 = openai_client.chat.completions.create(**request_2)
    
print(response_2.choices[0].message)


ChatCompletionMessage(content='\n', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_EOJbmXppQiqC1M7v6VnUzw', function=Function(arguments='{"query": "AI agent trends 2025 autonomous agents multi-agent systems", "topic": "general"}', name='internet_search'), type='function', index=0)], reasoning='\nGreat! I have some initial search results about AI agent trends for 2025. Let me analyze what I found:\n\n1. Data Society article mentions AI agents as the defining workforce trend of 2025, highlighting various workplace applications\n2. Microsoft Source article mentions 6 AI trends for 2025 including AI-powered agents with greater autonomy\n3. Warmly AI article shows market growth statistics (45.8% CAGR from 2025-2030)\n4. IBM has a comprehensive guide to AI agents\n5. Morgan Stanley article discusses 5 AI trends shaping innovation and ROI in 2025\n\nLet me get more detailed information about these tr